<a href="https://colab.research.google.com/github/mcarbel/mygooglecolabpublic/blob/main/ExcelCompare7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting started**

The document you are reading is not a static web page, but an interactive environment called a **Colab notebook** that lets you write and execute code.

For example, here is a **code cell** with a short Python script that computes a value, stores it in a variable, and prints the result:

# **Parameters Setup**
# path to files


In [8]:
import pandas as pd
from pathlib import Path

#define parameters
#path to files
path_old=Path(r'Quoted_Borrowings_valuation_ASK_30.06.2021_DEV_EOM_20210630_20210713_Copy.xlsx', sheet_name = 'Detailed', header = 0 , index_col=0)
path_new=Path(r'Quoted_Borrowings_valuation_ASK_30.06.2021_PROD_EOM_20210630_20210713_Copy.xlsx', sheet_name = 'Detailed', header = 0, index_col = 0)



# **Define Index**

In [35]:
#list of key column(s)
key=['Instrument','ISIN','Trn']
#sheets to read in
#sheet='Detailed'

In [40]:
# Read in the two excel files and fill NA
old = pd.read_excel(path_old).fillna(0)
new = pd.read_excel(path_new).fillna(0)


In [41]:
#set index
old=old.set_index(key)
new=new.set_index(key)

In [63]:
old=""

In [64]:
new=""

# **Dropped/Added Rows Dataframe**

In [44]:
#identify dropped rows and added (new) rows
dropped_rows = set(old.index) - set(new.index)
added_rows = set(new.index) - set(old.index)


In [62]:
dropped_rows

set()

In [61]:
added_rows

set()

In [65]:
old

In [66]:
new

# **Combine data for comparison**

In [55]:
#combine data
df_all_changes = pd.concat([old, new], axis='columns', keys=['old','new'], join='inner')

In [54]:


#prepare function for comparing old values and new values
def report_diff(x):
    return x[0] if x[0] == x[1] else '{} ---> {}'.format(*x)





In [56]:
#swap column indexes
df_all_changes = df_all_changes.swaplevel(axis='columns')[new.columns[0:]]

In [ ]:
#apply the report_diff function
df_changed = df_all_changes.groupby(level=0, axis=1).apply(lambda frame: frame.apply(report_diff, axis=1))

In [57]:
#create a list of text columns (int columns do not have '{} ---> {}')
df_changed_text_columns = df_changed.select_dtypes(include='object')


In [60]:
#create 3 datasets:
#diff - contains the differences
#dropped - contains the dropped rows
#added - contains the added rows
diff = df_changed_text_columns[df_changed_text_columns.apply(lambda x: x.str.contains("--->") == True, axis=1)]
dropped = old.loc[dropped_rows]
added = new.loc[added_rows]


#create a name for the output excel file
fname =  '{} vs {}.xlsx'.format(path_old.stem, path_new.stem)



# Write dataframe to excel

In [ ]:
#write dataframe to excel
writer=pd.ExcelWriter(fname, engine='xlsxwriter')
diff.to_excel(writer, sheet_name='diff', index=True)
dropped.to_excel(writer, sheet_name='dropped', index=True)
added.to_excel(writer, sheet_name='added', index=True)

In [ ]:
#get xlswriter objects
workbook = writer.book
worksheet = writer.sheets['diff']
worksheet.hide_gridlines(2)
worksheet.set_default_row(15)

In [ ]:
#get number of rows of the df diff
row_count_str=str(len(diff.index)+1)

In [ ]:
#define and apply formats
highligt_fmt = workbook.add_format({'font_color': '#FF0000', 'bg_color':'#B1B3B3'})
worksheet.conditional_format('A1:ZZ'+row_count_str, {'type':'text', 'criteria':'containing', 'value':'--->',
                            'format':highligt_fmt})


In [ ]:
#save the output
writer.save()
print ('\nDone.\n')